# ETL
This notebook is used to extract and transform the data from the CSV files in `/Datasets` and load them into the SQLite database.


In [1]:
# Dependencies
import pandas as pd
from pathlib import Path

In [2]:
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import desc

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean

## Import CSVs
#### Movies CSV and Omdb CSV (merged)
- Source: `Datasets/movies.csv` and `Datasets/omdb.csv`
- Table: `omdb_movies` in `Server/movies_db.sqlite`
#### Actor CSV
- Source: `Datasets/actor.csv`
- Table: `actor` in `Server/movies_db.sqlite`
#### Character CSV
- Source: `Datasets/character.csv`
- Table: `character` in `Server/movies_db.sqlite`

In [3]:
# Get the movies data
movies_csv = Path('../Datasets/movies.csv')
movies_df = pd.read_csv(movies_csv)

# Get the datatype from the DataFrame
movies_df.dtypes


movieid           int64
title            object
mpaa_rating      object
budget            int64
gross             int64
release_date     object
genre            object
runtime           int64
rating          float64
rating_count    float64
summary          object
dtype: object

In [4]:
# Get the omdb data
omdb_csv = Path('../Datasets/omdb.csv')
omdb_df = pd.read_csv(omdb_csv)

# Get the datatype from the DataFrame
omdb_df.dtypes

Title            object
Year             object
Rated            object
Released         object
Runtime          object
Genre            object
Director         object
Writer           object
Actors           object
Plot             object
Language         object
Country          object
Awards           object
Poster           object
Ratings          object
Metascore       float64
imdbRating      float64
imdbVotes        object
imdbID           object
Type             object
DVD              object
BoxOffice        object
Production       object
Website         float64
Response           bool
Error            object
totalSeasons    float64
dtype: object

In [5]:
omdb_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error,totalSeasons
0,Look Who's Talking,1989,PG-13,13 Oct 1989,93 min,Comedy,Amy Heckerling,Amy Heckerling,"John Travolta, Kirstie Alley, Olympia Dukakis","After a single, career-minded woman is left on...",...,"85,437",tt0097778,movie,12 Feb 2014,"$140,088,813",NaN,NaN,True,NaN,NaN
1,Driving Miss Daisy,1989,PG,26 Jan 1990,99 min,"Comedy, Drama, Romance",Bruce Beresford,Alfred Uhry,"Morgan Freeman, Jessica Tandy, Dan Aykroyd",An old Jewish woman and her African-American c...,...,"114,875",tt0097239,movie,03 Oct 2008,"$106,593,296",NaN,NaN,True,NaN,NaN
2,Turner & Hooch,1989,PG,28 Jul 1989,97 min,"Comedy, Crime, Drama",Roger Spottiswoode,"Dennis Shryack, Michael Blodgett, Daniel Petri...","Tom Hanks, Mare Winningham, Craig T. Nelson",A detective must adopt a rambunctious dog in o...,...,"74,257",tt0098536,movie,01 Jan 2014,"$71,079,915",NaN,NaN,True,NaN,NaN
3,Born on the Fourth of July,1989,R,05 Jan 1990,145 min,"Biography, Drama, War",Oliver Stone,"Ron Kovic, Oliver Stone","Tom Cruise, Bryan Larkin, Raymond J. Barry",The biography of Ron Kovic. Paralyzed in the V...,...,"113,083",tt0096969,movie,19 Apr 2016,"$70,001,698",NaN,NaN,True,NaN,NaN
4,Field of Dreams,1989,PG,05 May 1989,107 min,"Drama, Family, Fantasy",Phil Alden Robinson,"W.P. Kinsella, Phil Alden Robinson","Kevin Costner, James Earl Jones, Ray Liotta",Iowa farmer Ray Kinsella is inspired by a voic...,...,"124,247",tt0097351,movie,15 Jun 2012,"$64,431,625",NaN,NaN,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,Interstellar,2014,PG-13,07 Nov 2014,169 min,"Adventure, Drama, Sci-Fi",Christopher Nolan,"Jonathan Nolan, Christopher Nolan","Matthew McConaughey, Anne Hathaway, Jessica Ch...",When Earth becomes uninhabitable in the future...,...,"1,980,713",tt0816692,movie,24 May 2016,"$188,020,017",NaN,NaN,True,NaN,NaN
632,Captain America: The Winter Soldier,2014,PG-13,04 Apr 2014,136 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely, Joe Simon","Chris Evans, Samuel L. Jackson, Scarlett Johan...",As Steve Rogers struggles to embrace his role ...,...,"877,993",tt1843866,movie,19 Aug 2014,"$259,766,572",NaN,NaN,True,NaN,NaN
633,Dawn of the Planet of the Apes,2014,PG-13,11 Jul 2014,130 min,"Action, Adventure, Drama",Matt Reeves,"Mark Bomback, Rick Jaffa, Amanda Silver","Gary Oldman, Keri Russell, Andy Serkis",The fragile peace between apes and humans is t...,...,"455,909",tt2103281,movie,01 Dec 2015,"$208,545,589",NaN,NaN,True,NaN,NaN
634,Guardians of the Galaxy,2014,PG-13,01 Aug 2014,121 min,"Action, Adventure, Comedy",James Gunn,"James Gunn, Nicole Perlman, Dan Abnett","Chris Pratt, Vin Diesel, Bradley Cooper",A group of intergalactic criminals must pull t...,...,"1,241,602",tt2015381,movie,15 Nov 2015,"$333,718,600",NaN,NaN,True,NaN,NaN


In [6]:
# Remove rows with no data or irrelevant data
omdb_df.drop(omdb_df.loc[omdb_df['Error'] == 'Movie not found!'].index, inplace=True)
omdb_df.drop(omdb_df.loc[omdb_df['Type'] == 'series'].index, inplace=True)
omdb_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error,totalSeasons
0,Look Who's Talking,1989,PG-13,13 Oct 1989,93 min,Comedy,Amy Heckerling,Amy Heckerling,"John Travolta, Kirstie Alley, Olympia Dukakis","After a single, career-minded woman is left on...",...,"85,437",tt0097778,movie,12 Feb 2014,"$140,088,813",NaN,NaN,True,NaN,NaN
1,Driving Miss Daisy,1989,PG,26 Jan 1990,99 min,"Comedy, Drama, Romance",Bruce Beresford,Alfred Uhry,"Morgan Freeman, Jessica Tandy, Dan Aykroyd",An old Jewish woman and her African-American c...,...,"114,875",tt0097239,movie,03 Oct 2008,"$106,593,296",NaN,NaN,True,NaN,NaN
2,Turner & Hooch,1989,PG,28 Jul 1989,97 min,"Comedy, Crime, Drama",Roger Spottiswoode,"Dennis Shryack, Michael Blodgett, Daniel Petri...","Tom Hanks, Mare Winningham, Craig T. Nelson",A detective must adopt a rambunctious dog in o...,...,"74,257",tt0098536,movie,01 Jan 2014,"$71,079,915",NaN,NaN,True,NaN,NaN
3,Born on the Fourth of July,1989,R,05 Jan 1990,145 min,"Biography, Drama, War",Oliver Stone,"Ron Kovic, Oliver Stone","Tom Cruise, Bryan Larkin, Raymond J. Barry",The biography of Ron Kovic. Paralyzed in the V...,...,"113,083",tt0096969,movie,19 Apr 2016,"$70,001,698",NaN,NaN,True,NaN,NaN
4,Field of Dreams,1989,PG,05 May 1989,107 min,"Drama, Family, Fantasy",Phil Alden Robinson,"W.P. Kinsella, Phil Alden Robinson","Kevin Costner, James Earl Jones, Ray Liotta",Iowa farmer Ray Kinsella is inspired by a voic...,...,"124,247",tt0097351,movie,15 Jun 2012,"$64,431,625",NaN,NaN,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,Interstellar,2014,PG-13,07 Nov 2014,169 min,"Adventure, Drama, Sci-Fi",Christopher Nolan,"Jonathan Nolan, Christopher Nolan","Matthew McConaughey, Anne Hathaway, Jessica Ch...",When Earth becomes uninhabitable in the future...,...,"1,980,713",tt0816692,movie,24 May 2016,"$188,020,017",NaN,NaN,True,NaN,NaN
632,Captain America: The Winter Soldier,2014,PG-13,04 Apr 2014,136 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely, Joe Simon","Chris Evans, Samuel L. Jackson, Scarlett Johan...",As Steve Rogers struggles to embrace his role ...,...,"877,993",tt1843866,movie,19 Aug 2014,"$259,766,572",NaN,NaN,True,NaN,NaN
633,Dawn of the Planet of the Apes,2014,PG-13,11 Jul 2014,130 min,"Action, Adventure, Drama",Matt Reeves,"Mark Bomback, Rick Jaffa, Amanda Silver","Gary Oldman, Keri Russell, Andy Serkis",The fragile peace between apes and humans is t...,...,"455,909",tt2103281,movie,01 Dec 2015,"$208,545,589",NaN,NaN,True,NaN,NaN
634,Guardians of the Galaxy,2014,PG-13,01 Aug 2014,121 min,"Action, Adventure, Comedy",James Gunn,"James Gunn, Nicole Perlman, Dan Abnett","Chris Pratt, Vin Diesel, Bradley Cooper",A group of intergalactic criminals must pull t...,...,"1,241,602",tt2015381,movie,15 Nov 2015,"$333,718,600",NaN,NaN,True,NaN,NaN


In [7]:
# Change column name in movies_df to be able to merge on that column
movies_df = movies_df .rename(columns={"title":"Title"})
movies_df

,movieid,Title,mpaa_rating,budget,gross,release_date,genre,runtime,rating,rating_count,summary
0,1,Look Who's Talking,PG-13,7500000,296000000,12/10/1989,Romance,93,5.9,73638.0,"After a single, career-minded woman is left on..."
1,2,Driving Miss Daisy,PG,7500000,145793296,13/12/1989,Comedy,99,7.4,91075.0,An old Jewish woman and her African-American c...
2,3,Turner & Hooch,PG,13000000,71079915,28/07/1989,Crime,100,7.2,91415.0,"Det. Scott Turner (Tom Hanks) is an uptight, b..."
3,4,Born on the Fourth of July,R,14000000,161001698,20/12/1989,War,145,7.2,91415.0,The biography of Ron Kovic. Paralyzed in the V...
4,5,Field of Dreams,PG,15000000,84431625,21/04/1989,Drama,107,7.5,101702.0,"An Iowa corn farmer, hearing voices, interpret..."
...,...,...,...,...,...,...,...,...,...,...,...
631,507,Interstellar,PG-13,165000000,675120017,5/11/2014,Science Fiction,169,8.6,1343549.0,A team of explorers travel through a wormhole ...
632,508,Captain America: The Winter Soldier,PG-13,170000000,714766572,20/03/2014,Action,136,7.7,685903.0,As Steve Rogers struggles to embrace his role ...
633,509,Dawn of the Planet of the Apes,PG-13,170000000,710644566,26/06/2014,Science Fiction,130,7.6,395425.0,A growing nation of genetically evolved apes l...
634,510,Guardians of the Galaxy,PG-13,170000000,773328629,30/07/2014,Action,121,8.0,978222.0,A group of intergalactic criminals must pull t...


In [8]:
# Merge omdb_database with movies_db
omdb_movies_db = pd.merge(omdb_df, movies_df, on="Title" )

# Remove irrelevant columns
omdb_movies_db = omdb_movies_db[['movieid', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director','budget', 'gross',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production']]

omdb_movies_db = omdb_movies_db.rename(columns={"movieid" : "MovieID","budget":"Budget","gross":"GrossRevenue"})
omdb_movies_db

,MovieID,Title,Year,Rated,Released,Runtime,Genre,Director,Budget,GrossRevenue,...,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production
0,1,Look Who's Talking,1989,PG-13,13 Oct 1989,93 min,Comedy,Amy Heckerling,7500000,296000000,...,https://m.media-amazon.com/images/M/MV5BYWQzNG...,"[{'Source': 'Internet Movie Database', 'Value'...",51.0,5.9,"85,437",tt0097778,movie,12 Feb 2014,"$140,088,813",NaN
1,2,Driving Miss Daisy,1989,PG,26 Jan 1990,99 min,"Comedy, Drama, Romance",Bruce Beresford,7500000,145793296,...,https://m.media-amazon.com/images/M/MV5BMDE2ZT...,"[{'Source': 'Internet Movie Database', 'Value'...",81.0,7.3,"114,875",tt0097239,movie,03 Oct 2008,"$106,593,296",NaN
2,3,Turner & Hooch,1989,PG,28 Jul 1989,97 min,"Comedy, Crime, Drama",Roger Spottiswoode,13000000,71079915,...,https://m.media-amazon.com/images/M/MV5BMGYxNj...,"[{'Source': 'Internet Movie Database', 'Value'...",36.0,6.2,"74,257",tt0098536,movie,01 Jan 2014,"$71,079,915",NaN
3,4,Born on the Fourth of July,1989,R,05 Jan 1990,145 min,"Biography, Drama, War",Oliver Stone,14000000,161001698,...,https://m.media-amazon.com/images/M/MV5BMTgwND...,"[{'Source': 'Internet Movie Database', 'Value'...",75.0,7.2,"113,083",tt0096969,movie,19 Apr 2016,"$70,001,698",NaN
4,5,Field of Dreams,1989,PG,05 May 1989,107 min,"Drama, Family, Fantasy",Phil Alden Robinson,15000000,84431625,...,https://m.media-amazon.com/images/M/MV5BNzk5OW...,"[{'Source': 'Internet Movie Database', 'Value'...",57.0,7.5,"124,247",tt0097351,movie,15 Jun 2012,"$64,431,625",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,507,Interstellar,2014,PG-13,07 Nov 2014,169 min,"Adventure, Drama, Sci-Fi",Christopher Nolan,165000000,675120017,...,https://m.media-amazon.com/images/M/MV5BZjdkOT...,"[{'Source': 'Internet Movie Database', 'Value'...",74.0,8.7,"1,980,713",tt0816692,movie,24 May 2016,"$188,020,017",NaN
615,508,Captain America: The Winter Soldier,2014,PG-13,04 Apr 2014,136 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo",170000000,714766572,...,https://m.media-amazon.com/images/M/MV5BMzA2ND...,"[{'Source': 'Internet Movie Database', 'Value'...",70.0,7.8,"877,993",tt1843866,movie,19 Aug 2014,"$259,766,572",NaN
616,509,Dawn of the Planet of the Apes,2014,PG-13,11 Jul 2014,130 min,"Action, Adventure, Drama",Matt Reeves,170000000,710644566,...,https://m.media-amazon.com/images/M/MV5BMTgwOD...,"[{'Source': 'Internet Movie Database', 'Value'...",79.0,7.6,"455,909",tt2103281,movie,01 Dec 2015,"$208,545,589",NaN
617,510,Guardians of the Galaxy,2014,PG-13,01 Aug 2014,121 min,"Action, Adventure, Comedy",James Gunn,170000000,773328629,...,https://m.media-amazon.com/images/M/MV5BNDIzMT...,"[{'Source': 'Internet Movie Database', 'Value'...",76.0,8.0,"1,241,602",tt2015381,movie,15 Nov 2015,"$333,718,600",NaN


# ETL for actors data

In [9]:
# Get the actors data
actor_df = Path('../Datasets/actor.csv')
actor_df = pd.read_csv(actor_df)

# Get the datatype from the DataFrame
actor_df.dtypes

actorid            int64
name              object
date_of_birth     object
birth_city        object
birth_country     object
height_inches    float64
biography         object
gender            object
ethnicity         object
networth         float64
dtype: object

In [10]:
actor_df

,actorid,name,date_of_birth,birth_city,birth_country,height_inches,biography,gender,ethnicity,networth
0,2169,Aaron Eckhart,1968-03-12,Cupertino,USA,61.0,"Aaron Eckhart was born on March 12, 1968 in Cu...",Male,White,16000000.0
1,2204,Aasif Mandvi,1966-03-05,Bombay,India,68.0,"Aasif Mandvi was born on March 5, 1966 in Bomb...",Male,Indian,3000000.0
2,5,Abe Vigoda,1921-02-24,Brooklyn,USA,73.0,Abraham Charles Vigoda was an American actor k...,Male,White,10000000.0
3,1618,Abigail Breslin,1996-04-14,New York City,USA,61.0,Academy Award-nominated actress Abigail Bresli...,Female,White,8000000.0
4,1062,Abraham Benrubi,1969-10-04,Indianapolis,USA,79.0,"Abraham Benrubi was born on October 4, 1969 in...",Male,White,12000000.0
...,...,...,...,...,...,...,...,...,...,...
2594,1337,Ziyi Zhang,1979-02-09,Beijing,China,65.0,Ziyi Zhang is a Chinese actress and model. She...,NaN,NaN,NaN
2595,2566,Zoe Kravitz,1988-12-01,Los Angeles,USA,62.0,"Zoe Isabella Kravitz, the daughter of singer/a...",Female,NaN,NaN
2596,2232,Zoe Saldana,1978-06-19,Passaic,USA,66.0,"Zoe Saldana was born on June 19, 1978 in Passa...",Female,NaN,35000000.0
2597,2534,Zoey Vargas,NaN,NaN,NaN,NaN,"Zoey Vargas is an actress, known for Neighbors...",Female,NaN,NaN


# ETL for characters data

In [6]:
# Get the character data
characters_df = Path('../Datasets/character.csv')
characters_df = pd.read_csv(characters_df)

# Get the datatype from the DataFrame
characters_df.dtypes

movieid             int64
actorid             int64
character_name     object
creditorder         int64
pay               float64
screentime         object
dtype: object

In [3]:
# Get Base
Base = declarative_base()

In [13]:
omdb_movies_db.columns

Index(['MovieID', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre',
       'Director', 'Budget', 'GrossRevenue', 'Writer', 'Actors', 'Plot',
       'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore',
       'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice',
       'Production'],
      dtype='object')

In [14]:
omdb_movies_db.dtypes

MovieID           int64
Title            object
Year             object
Rated            object
Released         object
Runtime          object
Genre            object
Director         object
Budget            int64
GrossRevenue      int64
Writer           object
Actors           object
Plot             object
Language         object
Country          object
Awards           object
Poster           object
Ratings          object
Metascore       float64
imdbRating      float64
imdbVotes        object
imdbID           object
Type             object
DVD              object
BoxOffice        object
Production       object
dtype: object

# Add to database

In [15]:
# Create omdb class
class Omdb_Movies(Base):
    __tablename__ = 'omdb_movies'
    MovieID = Column(Integer, primary_key=True)
    Title = Column(String)
    Year = Column(Integer)
    Rated = Column(String)
    Released = Column(String)
    Runtime = Column(String)
    Genre = Column(String)
    Director = Column(String)  
    Budget = Column(Float)
    GrossRevenue = Column(Float)
    Writer = Column(String)
    Actors = Column(String)
    Plot = Column(String)
    Language = Column(String)
    Country = Column(String)
    Awards = Column(String)
    Poster = Column(String)
    Ratings = Column(String)
    Metascore = Column(Float)
    imdbRating = Column(Float)
    imdbVotes = Column(String)
    imdbID = Column(String)
    Type = Column(String)
    DVD = Column(String)
    BoxOffice = Column(String)
    Production = Column(String)


In [10]:
# Create actors class
class Actor(Base):
    __tablename__ = 'actor'
    actorid = Column(Integer, primary_key=True)
    name = Column(String)
    date_of_birth = Column(String)
    birth_city = Column(String)
    birth_country = Column(String)
    height_inches = Column(Float)
    biography = Column(String) 
    gender = Column(String)
    ethnicity = Column(String)
    networth = Column(Float)

In [5]:
# Create character class
class Character(Base):
    __tablename__ = 'character'
    id = Column(Integer, primary_key=True)
    movieid = Column(Integer)
    actorid = Column(Integer)
    character_name = Column(String)

In [11]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///../Server/movies_db.sqlite')

# Create the tables within the database
Base.metadata.create_all(engine)
# Start session
session = Session(bind=engine)

In [19]:
# Loop through omdb_movies DataFrame and retrieve data
for index, row in omdb_movies_db.iterrows():   
    MovieID = index
    Title = omdb_movies_db.loc[index,'Title']
    Year = int(omdb_movies_db.loc[index,'Year'])
    Rated = omdb_movies_db.loc[index,'Rated']
    Released = omdb_movies_db.loc[index,'Released']
    Runtime = omdb_movies_db.loc[index,'Runtime']
    Genre = omdb_movies_db.loc[index,'Genre']
    Director = omdb_movies_db.loc[index,'Director']
    Budget = omdb_movies_db.loc[index,'Budget']
    GrossRevenue = omdb_movies_db.loc[index,'GrossRevenue']
    Writer = omdb_movies_db.loc[index,'Writer']
    Actors = omdb_movies_db.loc[index,'Actors']
    Plot = omdb_movies_db.loc[index,'Plot']
    Language = omdb_movies_db.loc[index,'Language']
    Country = omdb_movies_db.loc[index,'Country']
    Awards = omdb_movies_db.loc[index,'Awards']
    Poster = omdb_movies_db.loc[index,'Poster']
    Ratings = omdb_movies_db.loc[index,'Ratings']
    Metascore = omdb_movies_db.loc[index,'Metascore']
    imdbRating = omdb_movies_db.loc[index,'imdbRating']
    imdbVotes = omdb_movies_db.loc[index,'imdbVotes']
    imdbID = omdb_movies_db.loc[index,'imdbID']
    Type = omdb_movies_db.loc[index,'Type']
    DVD = omdb_movies_db.loc[index,'DVD']
    BoxOffice = omdb_movies_db.loc[index,'BoxOffice']
    Production = omdb_movies_db.loc[index,'Production']


    # Add data to database
    session.add(Omdb_Movies(
        MovieID = MovieID,
        Title =  Title,
        Year =   Year,
        Rated =   Rated,
        Released =  Released,
        Runtime =  Runtime,
        Genre =   Genre,
        Director =  Director,
        Budget = Budget,
        GrossRevenue = GrossRevenue,
        Writer =   Writer,
        Actors =   Actors,
        Plot =   Plot,
        Language =  Language,
        Country =   Country,
        Awards =   Awards,
        Poster =   Poster,
        Ratings = Ratings,
        Metascore =  Metascore,
        imdbRating =  imdbRating,
        imdbVotes = imdbVotes,
        imdbID = imdbID,
        Type = Type,
        DVD =   DVD,
        BoxOffice =  BoxOffice,
        Production = Production
    ))
print(f"{len(omdb_movies_db)} rows ready for commit.")

619 rows ready for commit.


In [20]:
# Loop through actor DataFrame and retrieve data
for index, row in actor_df.iterrows():
    actorid = int(actor_df.loc[index,'actorid'])
    name = actor_df.loc[index,'name']
    date_of_birth = actor_df.loc[index,'date_of_birth']
    birth_city = actor_df.loc[index,'birth_city']
    birth_country = actor_df.loc[index,'birth_country']
    height_inches = actor_df.loc[index,'height_inches']
    biography = actor_df.loc[index,'biography']
    gender = actor_df.loc[index,'gender']
    ethnicity = actor_df.loc[index,'ethnicity']
    networth = actor_df.loc[index,'networth']
    
    # Add data to database
    session.add(Actor(
        actorid = actorid,
        name = name,
        date_of_birth = date_of_birth,
        birth_city = birth_city,
        birth_country = birth_country,
        height_inches = height_inches,
        biography = biography,
        gender = gender,
        ethnicity = ethnicity,
        networth = networth
    ))
  
print(f"{len(actor_df)} rows ready for commit.")

2599 rows ready for commit.


In [13]:
# Loop through characters DataFrame and retrieve data
for index, row in characters_df.iterrows():
    id = index
    movieid = int(characters_df.loc[index,'movieid'])
    actorid = int(characters_df.loc[index,'actorid'])
    character_name = characters_df.loc[index,'character_name']

    # Add data to database
    session.add(Character(
    id = id,
    movieid = movieid,
    actorid = actorid,
    character_name = character_name

    ))
print(f"{len(characters_df)} rows ready for commit.")

4319 rows ready for commit.


In [14]:
# Commit changes to session
session.commit()

In [15]:
# Close session
session.close()